# Web Components

En el capítulo del DOM hemos introducido gran parte de los conocimientos necesarios para este capítulo. No obstante, necesitamos haber trabajado con todos los demás conceptos y haber practicado con proyectos simples para entender la importancia de los `Web Components`. 

Si queremos hacer una SPA que maneje datos obtenidos del servidor y los muestre de manera reactiva, podemos seguir estos pasos:

* El router manda descargar los datos del servidor y los pasa a una función que los aplica a una plantilla. 
* El router se suscribe a un Observable de los datos del servidor y cada vez que llegan nuevos datos los renderiza con una plantilla.
* El router renderiza una plantilla con "placeholders", espera los datos del servidor y rellena la plantilla con esos datos. 
* ...

En cualquiera de estas opciones, hay problemas para volver a renderizar cuando hay datos nuevos, para mantener a raya a las suscripciones, para gestionar el estado de la aplicación o para mantenerla suficientemente desacoplada. También es complicado reutilizar esas funciones en otros proyectos.  

Los `Web Components` no son una tecnología diferente a las vistas en los capítulos anteriores, se trata de una definición de la W3C para estandarizar lo que comenzaba a ser un aspecto común de los frameworks e intentar hacer componentes reutilizables para todos. Estos componentes se pueden hacer a partir de las mejoras de ES6 y HTML5. En la definición que podemos encontrar en la web de MDN: https://developer.mozilla.org/en-US/docs/Web/API/Web_components, no incorpora la reactividad, pero nosotros vamos a incorporarla para hacerlos más útiles en el curso. 

> Llegado el momento de hacer Web Components desde 0 con "vanilla Javascript" nos podemos preguntar si no será mejor usar un framework. Intentemos resistir esa tentación, al menos para aprender, porque esos conocimientos nos permitirán entender profundamente cómo funciona, por ejemplo, Angular o React. 


## Conceptos iniciales

Se pueden crear componentes de muchas maneras, pero la definición oficial parte de la posibilidad de crear **Elementos personalizados** o `Custom Elements`. Esto se puede hacer porque Javascript permite heredar de `HTMLElement`: https://developer.mozilla.org/en-US/docs/Web/API/Web_components/Using_custom_elements. Luego veremos las posibilidades que hay con esto. 

Una de las necesidades cubiertas por los Web Components es la reutilización del código.  Los elementos personalizados pueden ser diferentes al resto de la aplicación y tener sus propios scripts y estilos. Estos pueden colisionar con una aplicación para la que no fueron creados desde el principio. Pero Javascript permite encapsular estos elementos en un DOM separado del principal. Esta técnica se llama `Shadow DOM`. 

Para crear elementos personalizados en el `Shadow DOM`, se pueden hacer programáticamente o a través del HTML con etiquetas como `<template>` o `<slot>`. 

Por consiguiente, un `Web Component` tiene una clase que hereda de `HTMLElement` registrada en el `CustomElementRegistry` para que pueda ser utilizada en cualquier parte. Este elemento tiene un `Shadow DOM` (opcional) para que su código y estilos no molesten al resto y se suele usar `<template>` para las plantillas. A continuación se puede usar como cualquier etiqueta. 

### Alternativas

Como todo en Javascript, es opcional usar Web components, pero es que también es opcional usar todas las técnicas anteriores. Si no hacemos un `Custom Element` no entra en la definición de lo que estamos tratando, pero se puede hacer una función que retorne un elemento con `Shadow DOM` y código personalizado que atiende a los eventos, descarga los datos, se suscribe a Observables... El problema es que perdemos las ventajas del ciclo de vida de los `HTMLElements`. 

Tampoco es necesario siempre hacer `Shadow DOM` si el código o estilo no va a interferir nunca. 

Por otro lado, el uso de `<template>` puede ser muy farragoso en algunos casos comparado con las `Template Literals`. En nuestro caso, haremos un uso combinado de los mismos para aprovechar las ventajas de ambas técnicas. 

## Custom Elements

Se trata de elementos HTML que tienen un comportamiento definido por el desarrollador. Una vez registrados, quedan disponibles en el navegador en esa página web.

Ya existen algunos creados como `HTMLImageElement` o `HTMLParagrafElement` que tienen un comportamiento extendido respecto al estándar de los elementos comunes. Pero a nosotros nos interesa crear nuevos desde cero. 

Para crearlo tan solo hay que extender la clase:

```javascript
class PopupInfo extends HTMLElement {
  constructor() {
    super();
  }
  // Element functionality written in here
}
```

En el constructor se inicializa el elemento con valores por defecto, registro de eventos, creación del `Shadow Root` y poco más. Dejaremos la creación de elementos hijos o añadir atributos a estos para después, en otras etapas del ciclo de vida. 



### Ciclo de vida

Los elementos personalizados cuentan con una serie de funciones que pueden ser implementadas y que son invocadas a lo largo de su ciclo de vida. Estas son:

* connectedCallback(): Se llama cada vez que se agrega el elemento al documento. Se recomienda configurar el elemento en este punto, mejor que en el constructor. 
* disconnectedCallback(): Se llama cada vez que se elimina el elemento del documento.
* adoptedCallback(): Se llama cada vez que se mueve el elemento a un nuevo documento.
* attributeChangedCallback(): se llama cuando se cambian, agregan, eliminan o reemplazan atributos. 

Podemos probar este código en la consola del navegador: 

```javascript
class MyCustomElement extends HTMLElement {
  static observedAttributes = ["class", "size"];

  constructor() {
    super();
      console.log("Constructor")
  }

  connectedCallback() {
    console.log("Custom element added to page.");
  }

  disconnectedCallback() {
    console.log("Custom element removed from page.");
  }

  adoptedCallback() {
    console.log("Custom element moved to new page.");
  }

  attributeChangedCallback(name, oldValue, newValue) {
    console.log(`Attribute ${name} has changed from ${oldValue} to ${newValue}.`);
  }
}

customElements.define("my-custom-element", MyCustomElement);
// La prueba
let customElement = document.createElement('my-custom-element');
console.log('Creado');
document.body.append(customElement);
console.log('Añadido');
customElement.classList.add('customClass','customClass2');
console.log('Cambio del atributo class');
customElement.remove();
console.log('Eliminado');
```

Además de esas funciones del ciclo de vida, se pueden añadir otras. Una típica puede ser la función `render()` que retorna el elemento redibujado. Esto es útil si queremos que el contenido pueda ser actualizado reactivamente. 

> **Elementos HTML nativos** (como `div`, `span`, etc.) no tienen métodos del ciclo de vida de los Web Components, por lo que asignar `connectedCallback` o otras manualmente no funcionará.

### Registrar el elemento personalizado

Como se puede ver en el ejemplo anterior, lo hemos registrado con:

```javascript
customElements.define("my-custom-element", MyCustomElement);
```

Es importante recalcar que el nombre es preciso que tenga **un guión (-)** en medio del nombre para que Javascript no los confunda.

### Usar el elemento personalizado

Se puede usar de forma normal como cualquier etiqueta HTML:

```html
<my-custom-element></my-custom-element>
```

## Shadow DOM

Como ya hemos visto en la introducción, el objetivo del `Shadow DOM` es encapsular el comportamiento del elemento personalizado. 

A partir de la raíz de DOM principal de la página web cuelgan todos los nodos de forma jerárquica. El Shadow DOM permite registrar árboles ocultos a partir de un nodo del árbol principal. Estos árboles empiezan por el `Shadow Root`. Esta raíz está unida al árbol principal mediante un nodo que actúa de `Shadow Host`. 

Se pueden crear de forma imperativa mediante Javascript:

```javascript
const host = document.querySelector("#host");
const shadow = host.attachShadow({ mode: "open" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM";
shadow.appendChild(span);
```

También se puede crear de forma declarativa mediante HTML:

```html
<div id="host">
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM</span>
  </template>
</div>
```

La segunda manera permite delegar la creación del `Shadow DOM` al servidor, al enviar este el HTML ya sea dinámica o estáticamente. 

Observemos que en los dos casos tenemos la palabra `open`, esta permite que el `Shadow DOM` sea accesible desde la web. Si está en `close`, se verá, pero no será accesible mediante selectores. 

Si no ponemos el `shadowrootmode` en el `template` no se verá, si lo ponemos, ya sea open o close, se verá pero será o no accesible. Esto es porque `template` tiene dos propósitos, sin especificar el `shadowrootmode` es una plantilla útil para ser clonada, pero pensada para ser renderizada. Si lo ponemos como en el ejemplo, estamos indicando que requemos que actúe de `shadow root`. 

Llegados a este punto, podemos poner esto en práctica copiando y pegando este código en la consola de cualquier web que tengamos abierta:

```javascript
const body = document.querySelector("body");
const host = document.createElement('div');
body.append(host);
const shadow = host.attachShadow({ mode: "closed" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM from Javascript";
shadow.append(span);

const host2 = document.createElement('div');
host2.innerHTML = `
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM from HTML</span>
  </template>
`;
body.append(host2);
```

> El segundo no se va a ver porque innerHTML no crea el `Shadow Root`. Este, por javascript, ha de ser creado explícitamente como en el primer ejemplo. No obstante, aunque no se vea, el template queda accesible y puede ser clonado para incorporarlo más adelante. 

No es posible que un `Element` que actua como `shadow host` tenga tanto shadow-root como elementos hijos en el árbol principal. No da error, pero no serán visibles. De la misma manera, cualquier `div` o otros elementos que contengan `<template>` se convierte automáticamente en un `shadow host`, por lo que no puede tener otras etiquetas normales dentro. 

los nodos descencientes del shadow-root no son accesibles mediante `document.querySelector`, por ejemplo. No obstante, son accesibles a través del atributo `shadowRoot`, el cual permite ejecutar `querySelector` internamente.

```javascript
host.shadowRoot.querySelector("span");
```

#### Estilos en shadow DOM

Para dar estilo a un shadow DOM se puede hacer modificando `.style...` en los elementos hijos, poniendo la etiqueta `<style>` y escribiendo el CSS ahí mismo o importando los ficheros CSS.

Los estilos generales no se aplican a los elementos dentro del `shadow root`.  En general es positivo que los estilos del DOM principal no afecten a los Web components. Esto los hace más robustos para poderlos reutilizar y que siempre se vean igual. También hace que sus estilos no puedan contaminar los de la web principal. No obstante, si estamos usando, por ejemplo, `Boostrap` tanto en el componente como en la web en general, podemos aplicar alguna de las técnicas que explicaremos a continuación:

Podemos usar un código como este en la creación del componente para que adopte las reglas del documento:

```javascript
const adoptedStyleSheets = new CSSStyleSheet();        
const rules = [...document.styleSheets].flatMap(s => [...s.cssRules].map(r => r.cssText)).join(' ');
adoptedStyleSheets.replace(rules);
shadow.adoptedStyleSheets = [adoptedStyleSheets];
```
Aquí se ha obtenido la lista de reglas de todas las hojas de estilo y se han añadido al `shadow`. Para ello hay que transformar mediante flatMap y demás la lista de estilos, dentro de ellos las reglas y dentro las reglas en string, crear un `CSSStyleSheet` personalizado y añadir todas las reglas. 

También se podrían buscar solo las reglas de un determinado tipo, una regla en concreto o las que vienen en un fichero. 

En el caso de usar `Vite y Bootstrap`, podemos importar el css con `?inline`, lo cual importa el texto del css y no lo inyecta en la web (https://vitejs.dev/guide/features#disabling-css-injection-into-the-page). Luego ese texto se puede usar tal cual en la plantilla. Observa el ejemplo:

```javascript
import * as bootstrap from 'bootstrap'
import './styles.scss'
import styles from "./styles.scss?inline"

const host = document.querySelector("#host");
const shadow = host.attachShadow({ mode: "open" });
const span = document.createElement("div");
span.innerHTML = `
 <style>${styles}</style>
<button type="button" class="btn btn-primary">Primary</button>
<button type="button" class="btn btn-secondary">Secondary</button>
<button type="button" class="btn btn-success">Success</button>
<button type="button" class="btn btn-danger">Danger</button>
<button type="button" class="btn btn-warning">Warning</button>
<button type="button" class="btn btn-info">Info</button>
<button type="button" class="btn btn-light">Light</button>
<button type="button" >Dark</button>
`;
shadow.appendChild(span);
```



## Templates

Ya hemos visto en el apartado anterior que se pueden crear `Shadow Root` desde una `<template>`. En los ejemplos anteriores eran usados con ese propósito, pero la etiqueta `<template>` también puede ser usada para hacer plantillas que no se van a ver hasta que se clonen.

Suponiendo que definimos esta plantilla:

```html
<template id="custom-paragraph">
  <p>My paragraph</p>
</template>
```

Podemos crear un `Web Component` de esta manera:

```javascript
customElements.define(
  "my-paragraph",
  class extends HTMLElement {
    constructor() {
      super();
      let template = document.getElementById("custom-paragraph");
      let templateContent = template.content;

      const shadowRoot = this.attachShadow({ mode: "open" });
      shadowRoot.appendChild(templateContent.cloneNode(true));
    }
  }
);
```

Observemos que estamos extendiendo la clase `HTMLElement` dentro de la misma función `customElements.define()`, que creamos explícitamente el `Shadow Root` y que le añadimos un clon de la `<template>`.

Es posible crear plantillas con `<template>` que puedan ser modificadas después de ser clonadas. Se puede hacer manualmente buscando dentro de ellas los elementos y cambiando su `innerText` o `value`. Por ejemplo, si tenemos que crear una tabla y poner datos en cada fila, podemos tener una plantilla para las filas y, en este caso, para la última columna:

```html
<template id="tableBodyTR">
  <tr>
    <th scope="row">{id}</th>
    <td>{ejemplo}</td>
  </tr>
</template>
<template id="lastColumn">
  <td class="buttonsCol">
    <span class="edit">📝</span><span class="delete">🗑️</span>
  </td>
</template>
```

```javascript
let lastColumnTemplate = div.querySelector("#lastColumn");
let tableBody = div.querySelector("#tableBody");
let tableBodyTR = div.querySelector("#tableBodyTR");

for (let row of datos) {
  let tableBodyTRRow = tableBodyTR.content.cloneNode(true).querySelector("tr");
  tableBodyTRRow.innerHTML = columns
    .map((col) => `<td data-col="${col}">${row[col]}</td>`)
    .join("");
  let lastColumn = lastColumnTemplate.content
    .cloneNode(true)
    .querySelector("td");
  lastColumn.dataset.id = row.id;
  tableBodyTRRow.append(lastColumn);
  tableBody.append(tableBodyTRRow);
}
```

En el ejemplo anterior, la primera columna de la plantilla `tableBodyTR` desaparece por el innerHTML. 

> Para usar `<template>` no es necesario estar en un `Web Component`. Se puede usar en cualquier momento, por eso también está explicado en el capítulo del DOM. 


## Pasar datos al componente

En los Web Components, a menudo hay que inicializarlos con algunos datos. Depende del propósito y cómo se añadan al DOM, se puede hacer de varias formas:

* Mediante atributos
* Mediante `data-`
* Mediante `slots`
* Mediante propiedades
* Mediante Observables (RxJS).

### Atributos
Los atributos se pueden usar directamente en el HTML del componente para pasar datos. Se acceden dentro del componente con this.getAttribute().

```javascript
<my-component nombre="Juan"></my-component>
...
  class MyComponent extends HTMLElement {
    connectedCallback() {
      const nombre = this.getAttribute('nombre');
      this.innerHTML = `<p>Hola, ${nombre}!</p>`;
    }
  }
  customElements.define('my-component', MyComponent);
```
> Hay que tener en cuenta que los atributos no son accesibles antes de que se añada al DOM. Por eso hay que acceder a ellos en en `connectedCallback`. 

###  `data-`

Es como los atributos pero más centralizado y accesible por `.dataset`. 

```javascript
<my-component data-nombre="María"></my-component>
...
  class MyComponent extends HTMLElement {
    connectedCallback() {
      const nombre = this.dataset.nombre;  // Accedemos con `dataset`
      this.innerHTML = `<p>Hola, ${nombre}!</p>`;
    }
  }
  customElements.define('my-component', MyComponent);
```

Problemas de pasar datos como atributos o en el dataset:
* No se pueden para más que datos primitivos como string.
* Si queremos pasar al componente un objeto complejo hay que serializarlo con `JSON.stringify()`. 
* Objetos complejos como Maps o Sets hay que convertirlos a Objects o Arrays antes de serializar. 
* Incluso si las string tienen comillas simples o dobles por el medio dan problemas tanto como atributos primitivos como dentro de objetos al serializar. 
* No es posible pasar funciones como atributos. 

### Propiedades

En este caso, los datos se pasan como propiedades de la instancia del componente en JavaScript. Las propiedades permiten una interacción más directa entre el componente y el código JavaScript que lo maneja.  Las propiedades no son visibles en el código HTML y pueden ser objetos más complejos que simples `strings` como, Sets, Maps, funciones o Observables. 

```javascript
<my-component></my-component>
...

  class MyComponent extends HTMLElement {
    set nombre(value) {
      this.innerHTML = `<p>Hola, ${value}!</p>`;
    }
  }
  customElements.define('my-component', MyComponent);

  const myComponent = document.querySelector('my-component');
  myComponent.nombre = 'Carlos';  // Establecemos la propiedad desde JS

```

En este ejemplo se usa un `Setter`, ya que el componente existirá antes de tener la propiedad. Pero si lo creamos por Javascript, podemos atender a esa propiedad en el `connectedCallback`.

### Observables

Utilitzando RxJS, se puede hacer que un componente sea reactivo a los cambios de datos pasados como flujos de eventos (Observables). Esto es útil para gestionar flujos asíncronos de datos.

```javascript
<my-component></my-component>
...
  class MyComponent extends HTMLElement {
    connectedCallback() {
      this.nombre$ = new BehaviorSubject('');  // Observable reactivo
      this.nombre$.subscribe(nombre => {
        this.innerHTML = `<p>Hola, ${nombre}!</p>`;
      });
    }

    set nombre(value) {
      this.nombre$.next(value);  // Emitimos el nuevo valor
    }
  }
  customElements.define('my-component', MyComponent);

  const myComponent = document.querySelector('my-component');
  myComponent.nombre = 'Ana';  // Actualizamos el observable

```

### Slots

Algunas veces se puede hacer más elegantemente con `<slot>`. La etiqueta debe tener un `name` y permite ser sustituida. 

Dentro de la plantilla se puede poner esta etiqueta de esta manera:

```html
<p><slot name="my-text">My default text</slot></p>
```

Y será incorporada en la plantilla en HTML así:

```html
<my-paragraph>
  <span slot="my-text">Let's have some different text!</span>
</my-paragraph>
```

### Alternativa con Template Literals

Usar `<template>` implica tener que buscarlo, clonarlo y modificarlo. En ocasiones es mucho más simple el uso de `Template Literals`. Incluso se puede hacer con `Tagged Template Literals` y queda un código más compacto. En el capítulo del DOM hay un ejemplo: https://xxjcaxx.github.io/libro_dwec/dom.html#creacion-de-elementos-mediante-tagged-template-literals 


## Ejemplo completo

Presuponemos un proyecto usando `Vite`, `Bootstrap` y `RxJS`.

La idea es crear un formulario reactivo como un `Web Component`. Para ello vamos a utilizar todas las técnicas explicadas, aunque alguna la podríamos omitir. En primer lugar, podemos crear una plantilla para el formulario con `<template>` y `<slot>`:

```javascript
const plantillaFormulario = `
    <div class="container mt-5">
      <h2 class="mb-4">
      <slot name="titulo">Formulario reactivo</slot>
      </h2>
      <form id="formularioReactivo">
        <!-- Campo de texto -->
        <div class="mb-3">
          <label for="nombre" class="form-label">Nombre</label>
          <input
            type="text"
            class="form-control"
            id="nombre"
            placeholder="Ingresa tu nombre"
            required
            name="nombre"
          />
        </div>

        <!-- Campo de correo electrónico -->
        <div class="mb-3">
          <label for="email" class="form-label">Correo Electrónico</label>
          <input
            type="email"
            class="form-control"
            id="email"
            placeholder="Ingresa tu correo electrónico"
            required
            name="correo"
          />
        </div>

        <!-- Botón de enviar -->
        <button type="submit" class="btn btn-primary">Enviar</button>
      </form>
      <div>
        <p><strong>Valor Nombre:</strong> 
        <span id="nombre-output"></span>
        </p>
        <p>
          <strong>Valor Correo Electrónico:</strong>
          <span id="email-output"></span>
        </p>
      </div>
    </div>
`;
```

En la tercera línea hay ` <slot name="titulo">Formulario reactivo</slot>` que luego será reemplazado. Por lo demás, es un formulario normal.

Para crear el `Web Component` vamos a usar una función. De esta manera, se puede reutilizar un poco más el código y se aproxima más a la programación funcional:

```javascript
const createCustomFormComponent = ({ // Object destructuring
  tag,
  template,
  keyUpFunction, 
}) => {
  class customComponent extends HTMLElement {
    constructor() {
      super();
      this.dataSubject = new Subject(); // La manera de obtener datos del padre
    }
    connectedCallback() {
      const templateComponent = document.createElement("template");
      templateComponent.innerHTML = template;
      const shadowRoot = this.attachShadow({ mode: "closed" });

      // Este paso es redundante e innecesario en este caso ya que estamos creando la plantilla dinámicamente
      // Lo dejamos para dejar la referencia a las funciones en un mismo ejemplo.
      const contentFragment = templateComponent.content;
      const content = contentFragment.cloneNode(true);
      shadowRoot.append(content);
    }
  }
  customElements.define(tag, customComponent);
};
```
En este caso es un componente para crear diversos formularios reactivos. Necesita para ser inicializado el `tag`, una plantilla del formulario que tenga inputs con el atributo `name` y una función para indicar qué hacer cuando ocurra un cambio en el formulario. 
Esta función ya registra el componente. 

Al `connectedCallback` le añadiremos las subscipciones necesarias para implementar la reactividad interna y externa: 


```javascript
connectedCallback() {
        const templateComponent = document.createElement('template');
        templateComponent.innerHTML = template;
        const shadowRoot = this.attachShadow({ mode: "closed" });

        // Este paso es redundante e innecesario en este caso ya que estamos creando la plantilla dinámicamente
        // Lo dejamos para dejar la referencia a las funciones en un mismo ejemplo. 
        const contentFragment = templateComponent.content;
        const content = contentFragment.cloneNode(true);
        shadowRoot.append(content);

        // El evento de keyup tratado de forma reactiva 
        this.keyUpSubscription = fromEvent(shadowRoot, 'keyup')
          .pipe(
            map((event) => new FormData(shadowRoot.querySelector('form'))),
            distinctUntilChanged(
              (previous, current) =>
                [...previous.entries()]
                  .every(
                    ([key, value], index) =>
                      value === [...current.entries()][index][1]
                  )),
            debounceTime(200)
          ).subscribe(values => {
            // Evento para informar al componente padre de un cambio
            const customEvent = new CustomEvent('formChanged', {
              bubbles: true,
              detail: { message: 'Form changed', values }
            });
            this.dispatchEvent(customEvent);
            // Implementar la reactividad en el mismo formulario
            keyUpFunction(shadowRoot)(values);
          });

          // Atender a los datos externos
        this.dataSubscription = this.dataSubject.subscribe(data => {
          [...shadowRoot.querySelectorAll('input')]
          .forEach(input => input.value = data[input.name]);
          shadowRoot.dispatchEvent(new KeyboardEvent('keyup'));
        });
      }
```

En este caso, nos interesa atender a los eventos `keyup` que modifiquen el formulario (`distinctUntilChanged`) con un cierto `debounceTime` para no saturar con cambios al componente padre. Cuando esto ocurre emitimos un evento personalizado, que es la manera de comunicar los cambios al componente padre. Luego se ejecuta la `keyUpFunction` tanto con el contenedor como con los últimos valores para que haga lo que se considere necesario. Esta función pasada como parámetros podría validad el formulario o, como en el caso que nos ocupa, mostrar los datos en otro sitio dentro del formulario. 

Es importante quitar todas las suscripciones a observables si se elimina el componente: 

```javascript
 disconnectedCallback() {
        this.keyUpSubscription.unsubscribe();
        this.dataSubscription.unsubscribe();
      }
```

Para crear el componente invocaremos a la función:

```javascript
  createCustomFormComponent(
    {  // Al pasar un objeto como parámetro, podemos asignar los nombres de los atributos y desordenarlos.
      tag: 'custom-formularioreactivo',
      template: plantillaFormulario,
      keyUpFunction: (shadowRoot) => (values) => {
        [...values].forEach(([key, value]) => {
          shadowRoot.querySelector(`#${key}-output`).innerText = value;
        });
      }
    }
  )
```

Como se puede ver, la `keyUpFunction` usará los valores del formulario para cambiar el contenido de ciertos elementos de la `plantillaFormulario`

Luego lo incorporamos a la web, por ejemplo:

```javascript
  const customFormulario = document.createElement('custom-formularioreactivo');

  // Crear el span con el slot="titulo"
  const spanTitulo = document.createElement('span');
  spanTitulo.setAttribute('slot', 'titulo');
  spanTitulo.textContent = 'Formulario reactivo con el titulo en slot';

  // Añadir el span al custom element
  customFormulario.appendChild(spanTitulo);

  // Insertar el custom element en el DOM
  document.querySelector("#reactividad").appendChild(customFormulario);
```

Ahora podemos comprobar que realmente es reactivo. Si modificamos inputs del formulario se ven los cambios. Pero también informa de los cambios al componente padre:

```javascript
  // Atender al evento 
  document.querySelector("#reactividad").addEventListener('formChanged', event => {
    console.log(event.detail);
  });
```
Con este código vemos que los cambios internos son devueltos como eventos personalizados.


Y el componente personalizado también se puede actualizar 
```javascript
  interval(1000).pipe(
    map(i=>({nombre: i, email: i}))
  )
  .subscribe(customFormulario.dataSubject)
```

Este código suscribe el subject interno del componente a un observable externo y esto provoca todos los cambios reactivamente. 

Lecturas:
* https://developer.mozilla.org/en-US/docs/Web/API/Web_components: Guia principal de referencia. 
* https://github.com/mdn/web-components-examples: Ejemplos de Custom Elements de la MDN.
* https://es.javascript.info/web-components Otro muy buen manual. 
* https://learn-wcs.com/?active-item=why-web-components Colección de enlaces muy buena.
* https://positive-intentions.com/blog/dim-functional-webcomponents/ Un ejemplo de programación funcional y reactiva con Web Components 